## Importing Libraries











































































































































































In [1]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlflow.models import infer_signature
from pprint import pprint
from typing import Union, Dict, List, Tuple
from datetime import datetime
import numpy as np
import pandas as pd
import yaml
import boto3
import joblib
import os
import math
import mlflow
import optuna
import warnings
import shutil
import ast

warnings.filterwarnings("ignore")

# Get the current datetime string for experiment names
current_datetime = datetime.now()
datetime_string = current_datetime.strftime("%Y-%m-%d_%H:%M:%S")

In [2]:
def read_yaml_file(path, file):
    # reading credentials files
    with open(f"{os.path.join(path, file)}") as f:
        try:
            content = yaml.safe_load(f)
        except yaml.YAMLError as e:
            raise e

    return content


CONFIG_PATH = os.path.join("src", "config")

In [ ]:
# credentials_config = read_yaml_file(path=CONFIG_PATH, file="credentials.yaml")

general_settings = read_yaml_file(path=CONFIG_PATH, file="settings.yaml")

# if credentials_config["EC2"] != "YOUR_EC2_INSTANCE_URL":
#     mlflow.set_tracking_uri(f"http://{credentials_config['EC2']}:5000")
# else:
#     mlflow.set_tracking_uri(f"http://mlflow:5000")

mlflow.set_tracking_uri(f"http://mlflow:5000")
print(f"Tracking Server URI: '{mlflow.get_tracking_uri()}'")

SEED = 42
ARTIFACTS_OUTPUT_PATH = general_settings["ARTIFACTS_PATH"]
FEATURES_OUTPUT_PATH = general_settings["FEATURES_PATH"]
RAW_FILE_PATH = os.path.join(
    general_settings["DATA_PATH"], general_settings["RAW_FILE_NAME"]
)
PROCESSED_RAW_FILE = "Preprocessed_" + general_settings["RAW_FILE_NAME"]
PROCESSED_RAW_FILE_PATH = os.path.join(
    general_settings["DATA_PATH"], PROCESSED_RAW_FILE
)
FEATURE_SELECTION_EXPERIMENT_NAME = f"feature-selection-experimentation"
HYPERPARAMETER_TUNING_EXPERIMENT_NAME = f"hyperparameters-tuning-experimentation"

FileNotFoundError: [Errno 2] No such file or directory: 'src/config/credentials.yaml'

## Loading Essentials

In [ ]:
# downloading the preprocessed dataset from the aws s3 bucket
if credentials_config["S3"] != "YOUR_S3_BUCKET_URL":
    # configuring AWS credentials
    os.environ["AWS_ACCESS_KEY_ID"] = credentials_config["AWS_ACCESS_KEY"]
    os.environ["AWS_SECRET_ACCESS_KEY"] = credentials_config["AWS_SECRET_KEY"]

    # downloading preprocessed dataset
    s3 = boto3.client(
        "s3",
        aws_access_key_id=credentials_config["AWS_ACCESS_KEY"],
        aws_secret_access_key=credentials_config["AWS_SECRET_KEY"]
    )
    s3.download_file(
        credentials_config["S3"],
        PROCESSED_RAW_FILE,
        PROCESSED_RAW_FILE_PATH
    )

    # downloading artifacts from the aws s3 bucket
    !aws s3 cp --recursive s3://{credentials_config["S3"]}/artifacts {ARTIFACTS_OUTPUT_PATH}

    # downloading models from the aws s3 bucket
    !aws s3 cp --recursive s3://{credentials_config["S3"]}/features {FEATURES_OUTPUT_PATH}

In [ ]:
# loading features
X_train = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "X_train.pkl"))
y_train = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "y_train.pkl"))

X_valid = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "X_valid.pkl"))
y_valid = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "y_valid.pkl"))

# loading artifacts
sc = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "features_sc.pkl"))
ohe = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "features_ohe.pkl"))
ohe_label = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "label_ohe.pkl"))

# loading feature columns
temp_df = pd.read_csv(PROCESSED_RAW_FILE_PATH, sep=",")
FEATURES_NAME = temp_df.columns.tolist()
del temp_df

# REMOVE THIS CELL IN ACTUAL EXECUTIONS
this is to make execution faster

In [ ]:
X_train = X_train[:1000, :10]
y_train = y_train[:1000]
X_valid = X_valid[:200, :10]
y_valid = y_valid[:200]

## Feature Selection Experimentation

In [ ]:
# creating the baseline models
dt = DecisionTreeClassifier(random_state=SEED)
rf = RandomForestClassifier(random_state=SEED, verbose=0, n_jobs=-1)
xg = XGBClassifier(random_state=SEED, n_jobs=-1)
lg = LGBMClassifier(random_state=SEED, verbose=-1, objective="multiclass")
cb = CatBoostClassifier(random_seed=SEED, verbose=0, allow_writing_files=False)

In [ ]:
def apply_feature_selection(
    model: Union[
        DecisionTreeClassifier,
        RandomForestClassifier,
        XGBClassifier,
        LGBMClassifier,
        CatBoostClassifier,
    ],
    number_features: int,
    X_train: np.ndarray,
    y_train: np.array,
    X_valid: np.ndarray,
    y_valid: np.array,
) -> Dict:
    # initializing and fitting the sfs class
    sfs = SequentialFeatureSelector(model, n_features_to_select=number_features, cv=3)
    sfs.fit(X=X_train, y=y_train)

    # getting the indexes of the best features
    selected_features_indexes = np.argwhere(sfs.get_support()).reshape(-1)

    reduced_X_train = sfs.transform(X_train)
    reduced_X_valid = sfs.transform(X_valid)

    # training the model
    model.fit(reduced_X_train, y_train)

    # calculating the training f1 score
    predicted_y_train = model.predict(reduced_X_train)
    train_f1 = f1_score(y_true=y_train, y_pred=predicted_y_train, average="weighted")

    # calculating the validation f1 score
    predicted_y_valid = model.predict(reduced_X_valid)
    valid_f1 = f1_score(y_true=y_valid, y_pred=predicted_y_valid, average="weighted")

    # inferring the signature of the trained model
    signature = infer_signature(
        model_input=reduced_X_train, model_output=predicted_y_train
    )

    # saving the metrics and artifacts that we want to log in mlflow
    selected_features_names = list(
        map(lambda i: FEATURES_NAME[i], selected_features_indexes.tolist())
    )

    results = {
        "train_f1": train_f1,
        "valid_f1": valid_f1,
        "features": selected_features_names,
        "model": model,
        "model_signature": signature,
    }

    return results


def set_configurations_mlflow(
    model: Union[
        DecisionTreeClassifier,
        RandomForestClassifier,
        XGBClassifier,
        LGBMClassifier,
        CatBoostClassifier,
    ],
    y_train: np.array,
    y_valid: np.array,
) -> Tuple[np.array, np.array, str, str]:
    # reshaping the target values (if needed) and setting the run name and which
    # flavor is being used for each machine learning model
    if isinstance(model, DecisionTreeClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "decision_tree"
        flavor = "sklearn"

    if isinstance(model, RandomForestClassifier):
        run_name = "random_forest"
        flavor = "sklearn"

    if isinstance(model, XGBClassifier):
        run_name = "xgboost"
        flavor = "xgboost"

    if isinstance(model, LGBMClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "lightgbm"
        flavor = "lightgbm"

    if isinstance(model, CatBoostClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "catboost"
        flavor = "catboost"

    # disabling some options of the current flavor's autolog
    if flavor == "sklearn":
        mlflow.sklearn.autolog(
            log_models=False,
            log_post_training_metrics=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "xgboost":
        mlflow.xgboost.autolog(
            log_models=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "lightgbm":
        mlflow.lightgbm.autolog(
            log_models=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "catboost":
        # there is no autolog implemented for catboost
        pass

    return y_train, y_valid, run_name, flavor


def run_feature_selection_experiment(
    models: List,
    min_features: int,
    max_features: int,
    experiment_id: str,
    metric_to_optimize: str = "valid_f1",
    direction: str = "max",
) -> None:
    for model in models:
        # reshaping the target values (if needed) and setting some mlflow's configuration
        new_y_train, new_y_valid, run_name, flavor = set_configurations_mlflow(
            model=model, y_train=y_train, y_valid=y_valid
        )
        model_type_results = []

        # starting a new run for the current model
        with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):
            pprint(f"Starting the run for the {run_name} model!\n")

            for i, n_features in enumerate(range(min_features, max_features + 1)):
                # creating a nested run inside the model's main run
                with mlflow.start_run(
                    experiment_id=experiment_id,
                    run_name=f"{run_name}_experiment_{i}",
                    nested=True,
                ) as run:
                    # running the feature selection main function
                    results = apply_feature_selection(
                        model=model,
                        number_features=n_features,
                        X_train=X_train,
                        y_train=new_y_train,
                        X_valid=X_valid,
                        y_valid=new_y_valid,
                    )

                    # logging the trained model
                    if flavor == "sklearn":
                        mlflow.sklearn.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model"s default parameters
                        mlflow.log_params(results["model"].get_params(deep=True))
                    elif flavor == "xgboost":
                        mlflow.xgboost.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_params(deep=True))
                    elif flavor == "lightgbm":
                        mlflow.lightgbm.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_params())
                    elif flavor == "catboost":
                        mlflow.catboost.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_all_params())

                    # logging the training and validation scores
                    mlflow.log_metric("train_f1", results["train_f1"])
                    mlflow.log_metric("valid_f1", results["valid_f1"])

                    # logging the artifacts (original dataset, features, and encoders objects)
                    mlflow.log_artifact(PROCESSED_RAW_FILE_PATH)
                    mlflow.log_artifact(ARTIFACTS_OUTPUT_PATH)
                    mlflow.log_artifact(FEATURES_OUTPUT_PATH)

                    # logging the indexes of the best features
                    mlflow.log_param("features", results["features"])

                    # add mlflow ids to results object and append to list
                    results["experiment_id"] = experiment_id
                    results["run_name"] = f"{run_name}_experiment_{i}"
                    results["flavor"] = flavor
                    results["run_id"] = run.info.run_id
                    model_type_results.append(results)

        # register the best feature selection in mlflow
        df = pd.DataFrame.from_records(model_type_results)
        if direction == "min":
            best_result = df.loc[df[metric_to_optimize].idxmin()]
        elif direction == "max":
            best_result = df.loc[df[metric_to_optimize].idxmax()]
        else:
            raise NotImplementedError("")
        tags = {"type": "baseline", "model": run_name}
        result = mlflow.register_model(
            model_uri=f"runs:/{best_result['run_id']}/{best_result['run_name']}",
            name=f"{FEATURE_SELECTION_EXPERIMENT_NAME}_{run_name}",
            tags=tags,
        )

In [ ]:
models = [dt, rf, xg, lg, cb]
min_features = math.floor(X_train.shape[1] * 0.2)
max_features = math.floor(X_train.shape[1] * 0.5)

# creating a new mlflow's experiment
experiment_id = mlflow.create_experiment(
    name=FEATURE_SELECTION_EXPERIMENT_NAME + "_" + datetime_string,
)

# running the feature selection experiments
run_feature_selection_experiment(
    models=models,
    min_features=min_features,
    max_features=max_features,
    experiment_id=experiment_id,
)

## Hyperparameters Tuning

In [ ]:
class Objective:
    def __init__(
        self,
        run_group_name: str,
        experiment_id: str,
        X_train: np.ndarray,
        y_train: np.array,
        X_valid: np.ndarray,
        y_valid: np.array,
        indexes: List,
    ) -> None:
        self.run_group_name = run_group_name
        self.experiment_id = experiment_id
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.indexes_name = indexes
        self.indexes = [FEATURES_NAME.index(i) for i in indexes]

        if self.run_group_name in ["decision_tree", "lightgbm", "catboost"]:
            self.y_train = np.argmax(self.y_train, axis=1)
            self.y_valid = np.argmax(self.y_valid, axis=1)

        self.X_train = self.X_train[:, self.indexes]
        self.X_valid = self.X_valid[:, self.indexes]

    def __call__(self, trial: optuna.trial.Trial) -> float:
        with mlflow.start_run(
            experiment_id=self.experiment_id,
            run_name=f"{self.run_group_name}_trial_{trial.number}",
            nested=True,
        ) as run:
            trial.set_user_attr("run_id", run.info.run_id)
            trial.set_user_attr("run_name", run.info.run_name)
            if self.run_group_name == "decision_tree":
                params = {
                    "max_depth": trial.suggest_int("max_depth", 2, 32, step=2),
                    "min_samples_split": trial.suggest_int(
                        "min_samples_split", 2, 8, step=1
                    ),
                    "min_samples_leaf": trial.suggest_int(
                        "min_samples_leaf", 1, 6, step=1
                    ),
                    "min_weight_fraction_leaf": trial.suggest_float(
                        "min_weight_fraction_leaf", 0, 0.5, step=0.1
                    ),
                    "max_leaf_nodes": trial.suggest_int(
                        "max_leaf_nodes", 2, 16, step=2
                    ),
                    "random_state": SEED,
                }
                model = DecisionTreeClassifier(**params)

            if self.run_group_name == "random_forest":
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
                    "max_depth": trial.suggest_int("max_depth", 10, 50),
                    "min_samples_split": trial.suggest_int("min_samples_split", 2, 32),
                    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 32),
                    "random_state": SEED,
                    "n_jobs": -1,
                }
                model = RandomForestClassifier(**params)

            if self.run_group_name == "xgboost":
                params = {
                    "booster": trial.suggest_categorical(
                        "booster", ["gbtree", "gblinear", "dart"]
                    ),
                    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
                    "random_state": SEED,
                    "n_jobs": -1,
                }
                model = XGBClassifier(**params)

            if self.run_group_name == "lightgbm":
                params = {
                    "objective": "multiclass",
                    "verbosity": -1,
                    "random_state": SEED,
                    "n_jobs": -1,
                    "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                    "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                    "feature_fraction": trial.suggest_float(
                        "feature_fraction", 0.4, 1.0
                    ),
                    "bagging_fraction": trial.suggest_float(
                        "bagging_fraction", 0.4, 1.0
                    ),
                    "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                    "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                }
                model = LGBMClassifier(**params)

            if self.run_group_name == "catboost":
                params = {
                    "random_seed": SEED,
                    "verbose": 0,
                    "allow_writing_files": False,
                    "colsample_bylevel": trial.suggest_float(
                        "colsample_bylevel", 0.01, 0.1
                    ),
                    "depth": trial.suggest_int("depth", 1, 12),
                    "boosting_type": trial.suggest_categorical(
                        "boosting_type", ["Ordered", "Plain"]
                    ),
                    "bootstrap_type": trial.suggest_categorical(
                        "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
                    ),
                }
                model = CatBoostClassifier(**params)

            model.fit(X=self.X_train, y=self.y_train)

            # calculating the training f1 score
            train_prediction = model.predict(self.X_train)
            train_f1 = f1_score(
                y_true=self.y_train, y_pred=train_prediction, average="weighted"
            )

            # calculating the validation f1 score
            valid_prediction = model.predict(self.X_valid)
            valid_f1 = f1_score(
                y_true=self.y_valid, y_pred=valid_prediction, average="weighted"
            )

            # logging the training and validation scores
            mlflow.log_metric("train_f1", train_f1)
            mlflow.log_metric("valid_f1", valid_f1)

            # inferring the signature of the trained model
            signature = infer_signature(
                model_input=self.X_train, model_output=train_prediction
            )

            # saving the trained model
            if self.run_group_name in ["decision_tree", "random_forest"]:
                # sklearn flavor
                mlflow.sklearn.log_model(
                    model, self.run_group_name, signature=signature
                )
                # logging the model"s default parameters
                mlflow.log_params(model.get_params(deep=True))
            elif self.run_group_name == "xgboost":
                mlflow.xgboost.log_model(
                    model, self.run_group_name, signature=signature
                )
                # logging the model's default parameters
                mlflow.log_params(model.get_params())
            elif self.run_group_name == "lightgbm":
                mlflow.lightgbm.log_model(
                    model, self.run_group_name, signature=signature
                )
                # logging the model's default parameters
                mlflow.log_params(model.get_params())
            elif self.run_group_name == "catboost":
                mlflow.catboost.log_model(
                    model, self.run_group_name, signature=signature
                )
                # logging the model's default parameters
                mlflow.log_params(model.get_all_params())

        return valid_f1

In [ ]:
# creating a new mlflow's experiment
hpt_experiment_id = mlflow.create_experiment(
    name=HYPERPARAMETER_TUNING_EXPERIMENT_NAME + "_" + datetime_string,
)

### Run optuna trials

In [ ]:
def get_latest_model_params(model_name):
    client = mlflow.tracking.MlflowClient()
    latest_version_info = client.get_latest_versions(model_name)[0]
    run_id = latest_version_info.run_id
    run = client.get_run(run_id)

    # Get the parameters logged for the run
    parameters = run.data.params
    return parameters


def run_mlflow_experiment(run_group_name, experiment_id, direction="maximize"):
    dt_run_params = get_latest_model_params(
        FEATURE_SELECTION_EXPERIMENT_NAME + "_" + run_group_name
    )
    dt_features_indexes = ast.literal_eval(dt_run_params["features"])

    with mlflow.start_run(experiment_id=experiment_id, run_name=run_group_name):
        objective = Objective(
            run_group_name=run_group_name,
            experiment_id=experiment_id,
            X_train=X_train,
            y_train=y_train,
            X_valid=X_valid,
            y_valid=y_valid,
            indexes=dt_features_indexes,
        )

        study = optuna.create_study(direction=direction)
        study.optimize(objective, n_trials=10)

        return study.trials_dataframe()


all_trials_dfs = []
run_group_names = ["decision_tree", "random_forest", "xgboost", "lightgbm", "catboost"]
for run_group_name in run_group_names:
    trials_df = run_mlflow_experiment(
        run_group_name, experiment_id, direction="maximize"
    )
    all_trials_dfs.append(trials_df)

### Analyze all trial results, and register best model

In [ ]:
df_trials = pd.concat(all_trials_dfs, axis=0)
best_trial_id = df_trials["value"].idxmax()
best_trial = df_trials.iloc[best_trial_id]

results = mlflow.register_model(
    model_uri=f"runs:/{best_trial['user_attrs_run_id']}/{best_trial['user_attrs_run_name']}",
    name=f"experimentation-best-model",
)

In [ ]:
if credentials_config["EC2"] != "YOUR_EC2_INSTANCE_URL":
    # removing downloaded dataset from local
    os.remove(PROCESSED_RAW_FILE_PATH)

    # removing the local artifacts and features
    shutil.rmtree(ARTIFACTS_OUTPUT_PATH)
    shutil.rmtree(FEATURES_OUTPUT_PATH)

In [9]:
import mlflow

logged_model = "runs:/464064cef6794b918e3b432f8587e674/decision_tree"

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [8]:
mlflow.set_tracking_uri(f"http://mlflow:5000")
model_uri = "models:/experimentation-best-model/4"
model = mlflow.pyfunc.load_model(model_uri=model_uri)

MlflowException: The following failures occurred while downloading one or more artifacts from http://mlflow:5000/api/2.0/mlflow-artifacts/artifacts/666474066399758848/464064cef6794b918e3b432f8587e674/artifacts/decision_tree_trial_2:
##### File  #####
API request to http://mlflow:5000/api/2.0/mlflow-artifacts/artifacts/666474066399758848/464064cef6794b918e3b432f8587e674/artifacts/decision_tree_trial_2/ failed with exception HTTPConnectionPool(host='mlflow', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/666474066399758848/464064cef6794b918e3b432f8587e674/artifacts/decision_tree_trial_2/ (Caused by ResponseError('too many 500 error responses'))